#  Blocking Cards Profiling

In [1]:
from common_imports import *
from data_const import *

/home/luca/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Notes

To understand how blocking cards work we have built a C program with libnfc that activates the rf field for a certain amount of seconds given as a parameter (default is 10s).

In [2]:
# !../apdu_smartcard_exchange_tester/magnetic_field_up # may need sudo

In this phase, data is recorded with GQRX. The waterfall plots are usefull to identify at what frquencies noise is generated. The data is presented in a real case scenario by moving the blocking card from far away, closer to the rtl-sdr whose antenna is above the reader. We can call this procedure 'blocking card' profiling.

In GQRX:
- FFT size (NFFT) is the number of samples to display in the waterfall/psd
- fps determins how quick the fft is shown
- samp_rate*time=window_size, time=window_size/samp_rate

Plots:
- Waterfall of a reader in presence of a blocking card (interstaff). 
- PSD. Note: The psd of thermal noise is constant in the frequency range of interest:
\begin{equation}
S_{x}(f) = \frac{N_{0}}{2}
\end{equation}
- Average of power (useful to detect when blocking cards starts to act)
- Max power: useful to detect at what frequency the blocking card oscillates (related to the distance from the reader)
- Probability density function of the signal. 
- Note: the pdf of wgn is a normal distribution

### Taking the magnitue of the signal

Produce the magnitude and store it (to view with urh) in case you have not recorded magintude with gnuradio

In [3]:
# for blck_name in blck_cards_list:
#     # load gqrx signal
#     data = load_gqrx("../data/gqrx_fixed/non-mag/%s.raw" % blck_name)
#     data = pd.DataFrame(data)
#     data["mag"] = np.sqrt(np.square(data[0]) + np.square(data[1]))
#     np.array(data["mag"]).tofile("../data/gqrx_fixed/mag/%s-mag.raw" % blck_name)

## Produce PDFs of blocking cards

In [6]:
for blck_name in blck_cards_list:
    # load gqrx signal
    if (not Path("imgs/profiling/fixed-pdf/%s.pdf" % blck_name).is_file()):
        data = load_mag("../data/gqrx_fixed/mag-trim/%s-mag.raw" % blck_name) + 0.3
        print(blck_name)
        plot_pdf(data, fig_name="fixed-pdf/%s.pdf" % blck_name)

## Produce Specgrams of blocking cards

In [7]:
from pathlib import Path
for blck_name in blck_cards_list:
    if (not Path("imgs/profiling/fixed-psd/%s.pdf" % blck_name).is_file()):
        # load gqrx signal
        data = load_mag("../data/gqrx_fixed/mag-trim/%s-mag.raw" % blck_name) + 0.3
        print(blck_name)
        spectrum = specgram(data, Fs=Fs_gqrx, NFFT=NFFT, fig_name="fixed-psd/%s.pdf" % blck_name)